# Lecture14 LLM時代のデータサイエンスとは

【2024年度版】

ChatGPTや大規模言語モデル(LLM)の元となったTransformerの概要やHuggingFaceを利用した実行例を試し、最後に、データサイエンスとの関係性についていくつか話題を実例を含めながら紹介する。

- 深層学習から発展し、ChatGPTなどのもとになった、Transformerについて、動かしてみる。（時系列データ予測もやってみる）
- LLMの活用例
 - CSVデータエージェント
 - Q&A with sources

【注意】実行環境は様々なライブラリのバージョンに依存します、将来の動作は全く保証されません。

## Transformer

ここでは「機械学習エンジニアのためのTransfomers」の例を実際に動かしてみます。

ここでは、HuggingFaceというオープンソースのモデルやデータをあつめたハブからpipeline()関数を使ってローカルにtransformerのモデルをダウンロード（実際にはcolabの仮想マシンに）して使っています。


https://github.com/nlp-with-transformers/notebooks/blob/main/01_introduction.ipynb

## Transformerの歴史
Transformerとは、深層学習のアーキテクチャの一つであり、Googleにより発明されました。それ以降、BERTやGPTなどが開発され、現在のGPT3.5やGPT4すなわちChatGPTのもとになるTransformerが開発されてきました。

![Transformerの歴史](https://raw.githubusercontent.com/nlp-with-transformers/notebooks/48e4a5e5c44b86e1593c0945a49af9675cfd7158//images/chapter01_timeline.png)

## 「エンコーダー・デコーダ」アーキテクチャ

深層学習の中で、RNN(再帰型ニューラルネットワーク）を用いて、入力文字の順序を持つ列を、状態にエンコードする、エンコーダーブロックと、状態からこれを、出力文字列に、やはりRNNを用いてでコードするデコードブロックを組み合わせたものが、トランスフォーマの基本となるアーキテクチャです。
![](https://raw.githubusercontent.com/nlp-with-transformers/notebooks/48e4a5e5c44b86e1593c0945a49af9675cfd7158//images/chapter01_enc-dec.png)

In [ ]:
!pip install diffusers accelerate scipy safetensors sacremoses

例題とするテキスト（英語）です、~~Amazon~~EC業者への問い合わせのメールのようです。

In [2]:
text = """Dear Amazon, last week I ordered an Optimus Prime action figure \
from your online store in Germany. Unfortunately, when I opened the package, \
I discovered to my horror that I had been sent an action figure of Megatron \
instead! As a lifelong enemy of the Decepticons, I hope you can understand my \
dilemma. To resolve the issue, I demand an exchange of Megatron for the \
Optimus Prime figure I ordered. Enclosed are copies of my records concerning \
this purchase. I expect to hear from you soon. Sincerely, Bumblebee."""

## テキスト分類の例
pipleline関数に、タスク種別を引数に指定すると、適切なモデルを自動選択して、モデルのインスタンスをcolab仮想マシンに生成します。（具体的なモデルを指定することもできます）。

In [ ]:
from transformers import pipeline
#distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)

classifier = pipeline("text-classification")

具体的に選択されたモデルを表示します。

In [ ]:
classifier.model.name_or_path

In [ ]:
classifier.model

このようにして、与えた文章に対して感情を分析してくれます。スコア付き。

In [ ]:
classifier(text)

一度、モデルを作れば、あとは何度も別のテキストを試せます。ディープラーニングの父、ヒントン教授のインタビューから

In [7]:
text_hinton="""Geoff Hinton

Yes, I do. I strongly believe that. I strongly believe that when we eventually understand how the brain works,
 that's going to give us lots of psychological insight too. Just as understanding chemistry at the atomic level,
 understanding of how molecules bump into each other and what happens,
 gives us lots of insight into the gas laws.
"""

In [ ]:
classifier(text_hinton)

## 固有語の識別

named entity(名前のあるエンティティ)の識別

In [ ]:
import pandas as pd
#dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf

ner_tagger = pipeline("ner", aggregation_strategy="simple")

outputs = ner_tagger(text)
pd.DataFrame(outputs)

## 質問に答える

In [ ]:

reader = pipeline("question-answering")
question = "What does the customer want?"
outputs = reader(question=question, context=text)
pd.DataFrame([outputs])

## 要約をする

In [ ]:

summarizer = pipeline("summarization")
outputs = summarizer(text, max_length=60, clean_up_tokenization_spaces=True)
print(outputs[0]['summary_text'])


## 翻訳では、FuguMTを使ってみます。

2024.5 なぜか壊れている。。。

→以下で対応してみた
`pip install -U transformers==4.30.2 `

In [ ]:
from transformers import pipeline

fugu_translator = pipeline('translation', model='staka/fugumt-en-ja')

#例(高慢と偏見 Jane Austen)
txt="""It is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife.
 However little known the feelings or views of such a man may be on his first entering a neighbourhood, this truth is so well
  fixed in the minds of the surrounding families, that he is considered the rightful property of some one or other of their daughters.
"""
fugu_translator(txt)

In [ ]:
fugu_translator(outputs[0]['summary_text'])

もう少し本格的な

In [ ]:
translator = pipeline("translation", model='Hoax0930/marian-finetuned-kde4-en-to-ja_kftt')
translator(outputs[0]['summary_text'])

In [ ]:
translator(txt)

## 画像生成
テキストから画像を生成する有名な、stablefusionも試せます。

In [ ]:
!pip install diffusers transformers accelerate scipy safetensors

メモリ利用量が無料版のcolabを超えるかもしれません。その場合は、クラッシュします。がメモリがリセットされるので再実行します。

In [ ]:
from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler
import torch

model_id = "stabilityai/stable-diffusion-2"

scheduler = EulerDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")
pipe = StableDiffusionPipeline.from_pretrained(model_id, scheduler=scheduler, torch_dtype=torch.float16)
pipe = pipe.to("cuda")

prompt = "The professor in panda outfits is teaching in a busy and packed class room. Some student is throwing spears to him"
image = pipe(prompt).images[0]
image

In [ ]:
prompt = "The professor in panda outfits is teaching in a busy and packed class room. Some student is throwing spears to him"
image = pipe(prompt).images[0]
image

## langchainで動かしてみます

In [ ]:
!pip install langchain langchain_community

まずは、ローカル環境(colab)にモデルをダウンロードしてローカルに使ってみます。

メモリ不足で、セッションがクラッシュする可能性もありますが、実行しなおせば大丈夫です汗

In [ ]:
from langchain_community.llms import HuggingFacePipeline
#from langchain import PromptTemplate,  LLMChain
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from IPython.display import display

model_id="bigscience/bloom-1b7"
llm = HuggingFacePipeline.from_model_id(model_id=model_id, task="text-generation", model_kwargs={"temperature":0, "max_length":64})

template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate(template=template, input_variables=["question"])

llm_chain = LLMChain(prompt=prompt, llm=llm)

question = "What is electroencephalography?"

answer = llm_chain.run(question)
display(answer)

In [ ]:
fugu_translator(answer)

一度でも、ロードが成功すれば、あとは大丈夫。

In [ ]:
question = "Why nuclear fusion is so difficult from enginnering perspective?"

display(llm_chain.run(question))

### OpenAIのapiのkeyを持っている場合は。OpenAIの計算リソースが利用できる

OpenAIの（超高速な）サーバーにあるモデルを使うことができます。ローカルメモリを消費しない。

In [ ]:
!pip install openai  langchain_openai

In [ ]:
# get a token: https://platform.openai.com/account/api-keys

from getpass import getpass

OPENAI_API_KEY = getpass()

··········


In [ ]:
import os

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [ ]:
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI

template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate(template=template, input_variables=["question"])

llm = OpenAI()

llm_chain = LLMChain(prompt=prompt, llm=llm)

question = "What is electroencephalography?"
answer = llm_chain.run(question)
display(answer)

" Firstly, we need to break down the word 'electroencephalography' into smaller parts. 'Electro' means electricity, 'encephalo' refers to the brain, and 'graphy' is the process of recording something. So, electroencephalography is the process of recording the electrical activity of the brain.\n\nIn simpler terms, electroencephalography (EEG) is a medical procedure that involves placing electrodes on a person's scalp to measure and record the electrical impulses produced by the brain. These electrodes detect and amplify the brain's electrical signals, which are then displayed as a graph or wave pattern on a computer screen. EEG is often used to diagnose and monitor conditions such as epilepsy, sleep disorders, and brain injuries. It can also be used in research to study brain function and activity."

In [ ]:
fugu_translator(answer)

[{'translation_text': 'まず「脳波」という言葉を より小さな部分に分解する必要があります 「エレクトロ」は電気 「脳」は脳を指し 「グラフ」は何かを記録するプロセスです エレクトロ脳波は脳の電気的活動を記録するプロセスです 簡単に言うと 脳波(EEG)とは脳が生み出す電気的インパルスを計測し記録する 電極を人の頭皮に貼り付ける医療手法です これらの電極は 脳の電気信号を検知して増幅し コンピュータのコンピュータに表示します 脳波は てんかん、睡眠障害、脳損傷などの状態を診断し監視するのによく使われます 脳機能や活動を研究する研究にも使われます'}]

## 有料のOpenAIのapi_keyを持っていなくても、無料のHugging Faceの計算リソースを使うことができます。

こちらも、HugginfFaceのもつ（たぶん、超高速の）サーバーを利用できます。

In [ ]:
!pip install -U from langchain-huggingface

ERROR: Could not find a version that satisfies the requirement from (from versions: none)
ERROR: No matching distribution found for from


In [ ]:
# get a token: https://huggingface.co/docs/api-inference/quicktour#get-your-api-token

from getpass import getpass

HUGGINGFACEHUB_API_TOKEN = getpass()

··········


In [ ]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

In [ ]:
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_community.llms import HuggingFaceEndpoint

template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate(template=template, input_variables=["question"])

repo_id = "mistralai/Mistral-7B-Instruct-v0.2"
llm = HuggingFaceEndpoint(repo_id=repo_id , temperature=0.5,  max_length=64)

llm_chain = LLMChain(prompt=prompt, llm=llm)

question = "What is electroencephalography?"
answer = llm_chain.run(question)
display(answer)

                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.


Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


' Electroencephalography, or EEG, is a medical test that measures the electrical activity of the brain. The word "electroencephalography" comes from three parts:\n\n1. "Electro": This refers to electricity, as the test measures electrical signals.\n2. "Encephalo": This is derived from the Greek word for brain.\n3. "Graphy": This means to write, as the test produces a graph or recording of the brain\'s electrical activity.\n\nSo, EEG is a test that records the electrical activity of the brain using electrodes placed on the scalp. These electrodes detect the tiny electrical impulses produced by neurons in the brain when they communicate with each other. The resulting EEG trace can provide valuable information about the brain\'s electrical activity, which can help diagnose various neurological and psychiatric conditions.'

In [ ]:
fugu_translator(answer)

[{'translation_text': '脳波(Electroencephalography、EEG)とは、脳の電気的活動を測定する医学的検査である。「脳波(Electroencephalography)」とは、電気的信号を測定するための電気のことを指す。「脳波(Encephalo)」とは、ギリシャ語で「脳」を意味する。「脳波(Graphy)」とは、脳が電気的活動のグラフや記録を作成するときに、脳が脳に電気的活動を記録することを意味している。つまり、脳波は、頭皮に電極を配置して脳の電気的活動を記録するテストである。これらの電極は、脳内のニューロンが互いに通信する際に生じる小さな電気的インパルスを検出する。結果として生じる脳波の痕跡は、脳の電気的活動に関する貴重な情報を提供し、様々な神経学的および精神医学的状態を診断するのに役立つ。'}]